In [3]:
from PIL import Image
from PIL.TiffTags import TAGS
import re
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.transform import Affine
import numpy as np 
import cv2
import imutils
import matplotlib.pyplot as plt



In [15]:
def metadata(imagen):
    img = Image.open(imagen)
# obtenemos los Tags de la metadata y se almacenan en un diccionario
    meta_dict = {TAGS[key]: img.tag[key] for key in img.tag_v2}
    # Se imprime el diccionario para obtener la composición de los datos
    # for rec in meta_dict:
    #     print(rec, ":", meta_dict[rec])

    # Extraemos el indicador xmp
    p = meta_dict.get("XMP")
    s = p.decode("UTF-8")
    # dividimos por el salto de linea y obtenemos una lista
    div = s.split("\n")

    #eliminamos los espacios vacios 

    for ind,recorrido in enumerate(div):
        div[ind]=recorrido.strip() 
    usar = div[17]

    result = re.search(":(.*)=", div[17])
    result.group(1)

    result2 = re.search("\"(.*)\"", div[17])
    result2.group(1)

    metadiccionario = {}
    for ind,recorrido in enumerate(div):
        try:
            metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
        except:
            pass

    metadiccionario = {}
    for ind,recorrido in enumerate(div):
        try:
            metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
        except:
            pass

    #print(metadiccionario)
    return metadiccionario

def TransfromRaster(img_path, img_ruta):
    metadiccionario = metadata(img_path)
    
    altura_vuelo = float(metadiccionario["RelativeAltitude"])
    distancia_focal = float(metadiccionario["CalibratedFocalLength"])

    resolucion = altura_vuelo/distancia_focal

    min_lon = (float(metadiccionario["GpsLongitude"])) - (
        float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
    max_lon = (float(metadiccionario["GpsLongitude"])) + (
        float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
    min_lat = (float(metadiccionario["GpsLatitude"])) - (
        float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
    max_lat = (float(metadiccionario["GpsLatitude"])) + (
        float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111

    img_data = rasterio.open(img_path, 'r')

    bands = [1] #Se especifica las cantidades de canales que tiene la imagen.
    count_bands = len(bands)
    data = img_data.read(bands)
    _, width, height = data.shape 
    crs = {'init': 'epsg:4326'}

    transform = rasterio.transform.from_bounds(
            min_lon, min_lat, max_lon, max_lat, height, width)
        
    with rasterio.open(
        img_ruta,  # Nombre del nuevo archivo georefenciado
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=count_bands,
        dtype="int8", #Tipo de dato, como la imagenes que se rotaron son de tipo int8, el tipo de escritura debe ser del mismo tipo, para evitar errores a la hora de procesar la imagen
        crs=crs,
        transform=transform,
        ) as dst:
            dst.write(data)

In [17]:
img_path = "Img/DJI_0011.TIF"
img_ruta = "Img"

TransfromRaster(img_path, img_path)
   


CPLE_AppDefinedError: Deleting Img/DJI_0011.TIF failed: Permission denied